# Регрессия: прогноз SI

Прямая регрессия значения SI (индекса избирательности), исключая признаки IC50 и CC50.

In [7]:
# Подготовка данных
X = df[features]
y = df["SI"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Обучение моделей
lr.fit(X_train_scaled, y_train)
y_pred_lr = lr.predict(X_test_scaled)

rf_grid.fit(X_train, y_train)
y_pred_rf = rf_grid.predict(X_test)

xgb_grid.fit(X_train, y_train)
y_pred_xgb = xgb_grid.predict(X_test)

lgb_grid.fit(X_train, y_train)
y_pred_lgb = lgb_grid.predict(X_test)

# Оценка
for name, preds in [("Linear Regression", y_pred_lr), ("Random Forest", y_pred_rf),
                    ("XGBoost", y_pred_xgb), ("LightGBM", y_pred_lgb)]:
    print(f"{name}: R2={r2_score(y_test, preds):.3f}, MAE={mean_absolute_error(y_test, preds):.2f}, MSE={mean_squared_error(y_test, preds):.2f}")


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001719 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 15084
[LightGBM] [Info] Number of data points in the train set: 638, number of used features: 166
[LightGBM] [Info] Start training from score 43.739511
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain

# Регрессия: прогноз индекса избирательности (SI)

В этом разделе выполняются следующие шаги:

1. **Формулировка задачи и подготовка данных**  
   - Целевая переменная — SI (Selectivity Index).  
   - Признаки — все дескрипторы `features`, без использования IC50 и CC50.  
   - Сплит данных: 80% обучение, 20% тест (`train_test_split`).  
   - Масштабирование признаков через `StandardScaler` для моделей, чувствительных к шкале.

2. **Обучение и настройка моделей**  
   - **LinearRegression** (без подбора гиперпараметров).  
   - **RandomForestRegressor** (GridSearchCV по `n_estimators=[100,300]`, `max_depth=[None,10]`).  
   - **XGBRegressor** (GridSearchCV по `n_estimators=[100,300]`, `max_depth=[3,6]`).  
   - **LGBMRegressor** (GridSearchCV по `n_estimators=[100,300]`, `max_depth=[-1,10]`).  

3. **Результаты на тестовой выборке**

| Модель               | R²            | MAE      | MSE             |
|----------------------|---------------|----------|-----------------|
| Linear Regression    | –19 932.992   | 14 418.08 | 40 269 129 767.29 |
| Random Forest        | 0.092         | 198.00   | 1 833 955.63    |
| XGBoost              | 0.090         | 197.09   | 1 838 085.87    |
| LightGBM             | 0.090         | 199.28   | 1 838 934.89    |

4. **Анализ результатов**  
   - **Linear Regression** продемонстрировала катастрофически низкое качество (отрицательное R²), т.к. прямая линейная зависимость не подходит для сложной задачи SI.  
   - **Ансамблевые методы** (Random Forest, XGBoost, LightGBM) объясняют лишь ~9% дисперсии SI (R² ≈ 0.09), что свидетельствует о слабой связи дескрипторов с этим показателем.  
   - Значения MAE (~198) и MSE (~1.8 млн) указывают на высокую ошибку прогноза, особенно из-за редких выбросов.

> **Вывод:** прямой регрессии SI через классические модели недостаточно — все алгоритмы показывают низкое качество (R² < 0.1). Для практических нужд лучше перейти к задаче классификации (разделение по порогу SI), где модели показывают более стабильные результаты.


In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from xgboost import XGBRegressor, XGBClassifier
from lightgbm import LGBMRegressor, LGBMClassifier
from sklearn.metrics import (
    r2_score, mean_absolute_error, mean_squared_error,
    accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
)
import warnings
warnings.filterwarnings("ignore")

# Загрузка и предобработка
df = pd.read_excel("Данные_для_курсовои_Классическое_МО.xlsx")
df.drop(columns=["Unnamed: 0"], inplace=True)
constant_cols = [c for c in df.columns if df[c].nunique() == 1]
df.drop(columns=constant_cols, inplace=True)
df.dropna(inplace=True)
features = [c for c in df.columns if c not in ["IC50, mM", "CC50, mM", "SI"]]
scaler = StandardScaler()

# Инициализация моделей (пример для регрессии IC50)
lr = LinearRegression()
rf = RandomForestRegressor(random_state=42)
rf_grid = GridSearchCV(rf, {"n_estimators":[100,300],"max_depth":[None,10]}, cv=5, scoring="r2")
xgb = XGBRegressor(random_state=42)
xgb_grid = GridSearchCV(xgb, {"n_estimators":[100,300],"max_depth":[3,6]}, cv=5, scoring="r2")
lgb = LGBMRegressor(random_state=42)
lgb_grid = GridSearchCV(lgb, {"n_estimators":[100,300],"max_depth":[-1,10]}, cv=5, scoring="r2")
